## ਐਮਬੈਡਿੰਗਸ

ਪਿਛਲੇ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ `vocab_size` ਦੀ ਲੰਬਾਈ ਵਾਲੇ ਉੱਚ-ਮਾਪਦੰਡ bag-of-words ਵੇਕਟਰਾਂ 'ਤੇ ਕੰਮ ਕੀਤਾ ਸੀ, ਅਤੇ ਅਸੀਂ ਖੁਦ ਹੀ ਘੱਟ-ਮਾਪਦੰਡ positional representation ਵੇਕਟਰਾਂ ਨੂੰ sparse one-hot representation ਵਿੱਚ ਬਦਲ ਰਹੇ ਸੀ। ਇਹ one-hot representation ਮੈਮਰੀ-ਅਸਰਦਾਰ ਨਹੀਂ ਹੈ, ਇਸ ਤੋਂ ਇਲਾਵਾ, ਹਰ ਸ਼ਬਦ ਨੂੰ ਇੱਕ ਦੂਜੇ ਤੋਂ ਅਲੱਗ ਮੰਨਿਆ ਜਾਂਦਾ ਹੈ, ਜ਼ਿਆਦਾ one-hot encoded ਵੇਕਟਰ ਸ਼ਬਦਾਂ ਦੇ ਵਿਚਕਾਰ ਕੋਈ semantic similarity ਨਹੀਂ ਦਿਖਾਉਂਦੇ।

ਇਸ ਯੂਨਿਟ ਵਿੱਚ, ਅਸੀਂ **News AG** ਡਾਟਾਸੈਟ ਦੀ ਖੋਜ ਜਾਰੀ ਰੱਖਾਂਗੇ। ਸ਼ੁਰੂ ਕਰਨ ਲਈ, ਆਓ ਡਾਟਾ ਲੋਡ ਕਰੀਏ ਅਤੇ ਪਿਛਲੇ ਨੋਟਬੁੱਕ ਤੋਂ ਕੁਝ ਪਰਿਭਾਸ਼ਾਵਾਂ ਪ੍ਰਾਪਤ ਕਰੀਏ।


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## ਐਮਬੈਡਿੰਗ ਕੀ ਹੈ?

**ਐਮਬੈਡਿੰਗ** ਦਾ ਮਤਲਬ ਹੈ ਸ਼ਬਦਾਂ ਨੂੰ ਘੱਟ-ਡਾਈਮੇਂਸ਼ਨਲ ਡੈਂਸ ਵੈਕਟਰਾਂ ਨਾਲ ਦਰਸਾਉਣਾ, ਜੋ ਕਿਸੇ ਤਰ੍ਹਾਂ ਸ਼ਬਦ ਦੇ ਅਰਥ ਨੂੰ ਪ੍ਰਗਟ ਕਰਦੇ ਹਨ। ਅਸੀਂ ਬਾਅਦ ਵਿੱਚ ਚਰਚਾ ਕਰਾਂਗੇ ਕਿ ਮਤਲਬਪੂਰਨ ਸ਼ਬਦ ਐਮਬੈਡਿੰਗਜ਼ ਕਿਵੇਂ ਬਣਾਈਆਂ ਜਾਂਦੀਆਂ ਹਨ, ਪਰ ਫਿਲਹਾਲ ਚਲੋ ਐਮਬੈਡਿੰਗਜ਼ ਨੂੰ ਇੱਕ ਤਰੀਕੇ ਵਜੋਂ ਸੋਚੀਏ ਜੋ ਸ਼ਬਦ ਵੈਕਟਰ ਦੀ ਡਾਈਮੇਂਸ਼ਨਲਟੀ ਨੂੰ ਘਟਾਉਂਦਾ ਹੈ।

ਇਸ ਤਰ੍ਹਾਂ, ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਇੱਕ ਸ਼ਬਦ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਲਵੇਗਾ ਅਤੇ ਨਿਰਧਾਰਤ `embedding_size` ਦਾ ਇੱਕ ਆਉਟਪੁਟ ਵੈਕਟਰ ਤਿਆਰ ਕਰੇਗਾ। ਇੱਕ ਅਰਥ ਵਿੱਚ, ਇਹ `Linear` ਲੇਅਰ ਦੇ ਕਾਫ਼ੀ ਮਿਲਦਾ-ਜੁਲਦਾ ਹੈ, ਪਰ ਇਹ ਇੱਕ-ਹਾਟ ਐਨਕੋਡ ਕੀਤੇ ਵੈਕਟਰ ਦੀ ਥਾਂ ਸ਼ਬਦ ਨੰਬਰ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਲੈ ਸਕੇਗਾ।

ਜਦੋਂ ਅਸੀਂ ਆਪਣੇ ਨੈੱਟਵਰਕ ਵਿੱਚ ਪਹਿਲੇ ਲੇਅਰ ਵਜੋਂ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ, ਤਾਂ ਅਸੀਂ ਬੈਗ-ਆਫ-ਵਰਡਜ਼ ਤੋਂ **ਐਮਬੈਡਿੰਗ ਬੈਗ** ਮਾਡਲ ਵਿੱਚ ਬਦਲ ਸਕਦੇ ਹਾਂ, ਜਿੱਥੇ ਅਸੀਂ ਪਹਿਲਾਂ ਆਪਣੇ ਪਾਠ ਵਿੱਚ ਹਰ ਸ਼ਬਦ ਨੂੰ ਉਸਦੇ ਸੰਬੰਧਿਤ ਐਮਬੈਡਿੰਗ ਵਿੱਚ ਬਦਲਦੇ ਹਾਂ, ਅਤੇ ਫਿਰ ਸਾਰੇ ਐਮਬੈਡਿੰਗਜ਼ 'ਤੇ ਕੁਝ ਸਮੂਹ ਫੰਕਸ਼ਨ ਗਣਨਾ ਕਰਦੇ ਹਾਂ, ਜਿਵੇਂ ਕਿ `sum`, `average` ਜਾਂ `max`।

![ਪੰਜ ਕ੍ਰਮ ਸ਼ਬਦਾਂ ਲਈ ਐਮਬੈਡਿੰਗ ਕਲਾਸੀਫਾਇਰ ਦਿਖਾਉਂਦੀ ਇੱਕ ਚਿੱਤਰ।](../../../../../translated_images/pa/embedding-classifier-example.b77f021a7ee67eee.webp)

ਸਾਡਾ ਕਲਾਸੀਫਾਇਰ ਨਿਊਰਲ ਨੈੱਟਵਰਕ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਨਾਲ ਸ਼ੁਰੂ ਹੋਵੇਗਾ, ਫਿਰ ਐਗਰੀਗੇਸ਼ਨ ਲੇਅਰ, ਅਤੇ ਇਸਦੇ ਉੱਪਰ ਲੀਨਿਅਰ ਕਲਾਸੀਫਾਇਰ:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### ਵੱਖ-ਵੱਖ ਲੰਬਾਈ ਵਾਲੀਆਂ ਲੜੀਆਂ ਨਾਲ ਨਿਪਟਣਾ

ਇਸ ਆਰਕੀਟੈਕਚਰ ਦੇ ਨਤੀਜੇ ਵਜੋਂ, ਸਾਡੇ ਨੈੱਟਵਰਕ ਲਈ ਮਿਨੀਬੈਚਜ਼ ਨੂੰ ਇੱਕ ਖਾਸ ਤਰੀਕੇ ਨਾਲ ਬਣਾਉਣਾ ਪਵੇਗਾ। ਪਿਛਲੇ ਯੂਨਿਟ ਵਿੱਚ, ਜਦੋਂ ਬੈਗ-ਆਫ-ਵਰਡਜ਼ ਵਰਤ ਰਹੇ ਸਨ, ਤਾਂ ਮਿਨੀਬੈਚ ਵਿੱਚ ਸਾਰੇ BoW ਟੈਂਸਰਾਂ ਦਾ ਆਕਾਰ `vocab_size` ਦੇ ਬਰਾਬਰ ਹੁੰਦਾ ਸੀ, ਭਾਵੇਂ ਸਾਡੇ ਟੈਕਸਟ ਲੜੀ ਦੀ ਅਸਲ ਲੰਬਾਈ ਜੋ ਮਰਜ਼ੀ ਹੋਵੇ। ਜਦੋਂ ਅਸੀਂ ਵਰਡ ਐਮਬੈਡਿੰਗਜ਼ ਵੱਲ ਜਾਵਾਂਗੇ, ਤਾਂ ਹਰ ਟੈਕਸਟ ਨਮੂਨੇ ਵਿੱਚ ਸ਼ਬਦਾਂ ਦੀ ਗਿਣਤੀ ਵੱਖ-ਵੱਖ ਹੋਵੇਗੀ, ਅਤੇ ਜਦੋਂ ਅਸੀਂ ਉਹਨਾਂ ਨਮੂਨਿਆਂ ਨੂੰ ਮਿਨੀਬੈਚਜ਼ ਵਿੱਚ ਜੋੜਾਂਗੇ, ਤਾਂ ਸਾਨੂੰ ਕੁਝ ਪੈਡਿੰਗ ਲਾਗੂ ਕਰਨੀ ਪਵੇਗੀ।

ਇਹ ਕੰਮ `collate_fn` ਫੰਕਸ਼ਨ ਨੂੰ ਡੇਟਾਸੋਰਸ ਵਿੱਚ ਪ੍ਰਦਾਨ ਕਰਕੇ ਕੀਤਾ ਜਾ ਸਕਦਾ ਹੈ:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### ਐਮਬੈਡਿੰਗ ਕਲਾਸੀਫਾਇਰ ਦੀ ਟ੍ਰੇਨਿੰਗ

ਹੁਣ ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਡਾਟਾਲੋਡਰ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰ ਲਿਆ ਹੈ, ਅਸੀਂ ਪਿਛਲੇ ਯੂਨਿਟ ਵਿੱਚ ਪਰਿਭਾਸ਼ਿਤ ਕੀਤੇ ਗਏ ਟ੍ਰੇਨਿੰਗ ਫੰਕਸ਼ਨ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਮਾਡਲ ਨੂੰ ਟ੍ਰੇਨ ਕਰ ਸਕਦੇ ਹਾਂ:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **ਨੋਟ**: ਅਸੀਂ ਇੱਥੇ ਸਿਰਫ 25,000 ਰਿਕਾਰਡਾਂ ਲਈ ਟ੍ਰੇਨਿੰਗ ਕਰ ਰਹੇ ਹਾਂ (ਇੱਕ ਪੂਰੇ epoch ਤੋਂ ਘੱਟ) ਸਮੇਂ ਦੀ ਬਚਤ ਲਈ, ਪਰ ਤੁਸੀਂ ਟ੍ਰੇਨਿੰਗ ਜਾਰੀ ਰੱਖ ਸਕਦੇ ਹੋ, ਕਈ epoch ਲਈ ਟ੍ਰੇਨ ਕਰਨ ਲਈ ਇੱਕ ਫੰਕਸ਼ਨ ਲਿਖ ਸਕਦੇ ਹੋ, ਅਤੇ ਉੱਚ ਸਹੀਤਾ ਹਾਸਲ ਕਰਨ ਲਈ ਲਰਨਿੰਗ ਰੇਟ ਪੈਰਾਮੀਟਰ ਨਾਲ ਪ੍ਰਯੋਗ ਕਰ ਸਕਦੇ ਹੋ। ਤੁਸੀਂ ਲਗਭਗ 90% ਸਹੀਤਾ ਤੱਕ ਪਹੁੰਚ ਸਕਦੇ ਹੋ।


### ਐਮਬੈਡਿੰਗਬੈਗ ਲੇਅਰ ਅਤੇ ਵੱਖ-ਵੱਖ ਲੰਬਾਈ ਵਾਲੇ ਕ੍ਰਮ ਦੀ ਪ੍ਰਤੀਨਿਧਤਾ

ਪਿਛਲੀ ਆਰਕੀਟੈਕਚਰ ਵਿੱਚ, ਸਾਨੂੰ ਸਾਰੇ ਕ੍ਰਮਾਂ ਨੂੰ ਇੱਕੋ ਲੰਬਾਈ ਵਿੱਚ ਪੈਡ ਕਰਨਾ ਪੈਂਦਾ ਸੀ ਤਾਂ ਜੋ ਉਹਨਾਂ ਨੂੰ ਇੱਕ ਮਿਨੀਬੈਚ ਵਿੱਚ ਫਿੱਟ ਕੀਤਾ ਜਾ ਸਕੇ। ਇਹ ਵੱਖ-ਵੱਖ ਲੰਬਾਈ ਵਾਲੇ ਕ੍ਰਮਾਂ ਨੂੰ ਪ੍ਰਤੀਨਿਧਤ ਕਰਨ ਦਾ ਸਭ ਤੋਂ ਕੁਸ਼ਲ ਤਰੀਕਾ ਨਹੀਂ ਹੈ - ਇੱਕ ਹੋਰ ਤਰੀਕਾ **ਆਫਸੈਟ** ਵੇਕਟਰ ਦੀ ਵਰਤੋਂ ਕਰਨਾ ਹੋਵੇਗਾ, ਜੋ ਇੱਕ ਵੱਡੇ ਵੇਕਟਰ ਵਿੱਚ ਸਟੋਰ ਕੀਤੇ ਸਾਰੇ ਕ੍ਰਮਾਂ ਦੇ ਆਫਸੈਟ ਨੂੰ ਰੱਖੇਗਾ।

![ਆਫਸੈਟ ਕ੍ਰਮ ਦੀ ਪ੍ਰਤੀਨਿਧਤਾ ਦਿਖਾਉਂਦੀ ਤਸਵੀਰ](../../../../../translated_images/pa/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: ਉੱਪਰ ਦਿੱਤੀ ਤਸਵੀਰ ਵਿੱਚ, ਅਸੀਂ ਅੱਖਰਾਂ ਦੇ ਕ੍ਰਮ ਨੂੰ ਦਿਖਾਇਆ ਹੈ, ਪਰ ਸਾਡੇ ਉਦਾਹਰਨ ਵਿੱਚ ਅਸੀਂ ਸ਼ਬਦਾਂ ਦੇ ਕ੍ਰਮਾਂ ਨਾਲ ਕੰਮ ਕਰ ਰਹੇ ਹਾਂ। ਹਾਲਾਂਕਿ, ਆਫਸੈਟ ਵੇਕਟਰ ਨਾਲ ਕ੍ਰਮਾਂ ਦੀ ਪ੍ਰਤੀਨਿਧਤਾ ਕਰਨ ਦਾ ਆਮ ਸਿਧਾਂਤ ਇੱਕੋ ਜਿਹਾ ਰਹਿੰਦਾ ਹੈ।

ਆਫਸੈਟ ਪ੍ਰਤੀਨਿਧਤਾ ਨਾਲ ਕੰਮ ਕਰਨ ਲਈ, ਅਸੀਂ [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) ਲੇਅਰ ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਹਾਂ। ਇਹ `Embedding` ਦੇ ਸਮਾਨ ਹੈ, ਪਰ ਇਹ ਸਮੱਗਰੀ ਵੇਕਟਰ ਅਤੇ ਆਫਸੈਟ ਵੇਕਟਰ ਨੂੰ ਇਨਪੁਟ ਵਜੋਂ ਲੈਂਦਾ ਹੈ, ਅਤੇ ਇਸ ਵਿੱਚ ਇੱਕ ਐਵਰੇਜਿੰਗ ਲੇਅਰ ਵੀ ਸ਼ਾਮਲ ਹੁੰਦੀ ਹੈ, ਜੋ ਕਿ `mean`, `sum` ਜਾਂ `max` ਹੋ ਸਕਦੀ ਹੈ।

ਹੇਠਾਂ ਇੱਕ ਸੋਧਿਆ ਹੋਇਆ ਨੈਟਵਰਕ ਦਿੱਤਾ ਗਿਆ ਹੈ ਜੋ `EmbeddingBag` ਦੀ ਵਰਤੋਂ ਕਰਦਾ ਹੈ:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

ਡਾਟਾਸੈੱਟ ਨੂੰ ਟ੍ਰੇਨਿੰਗ ਲਈ ਤਿਆਰ ਕਰਨ ਲਈ, ਸਾਨੂੰ ਇੱਕ ਰੂਪਾਂਤਰਨ ਫੰਕਸ਼ਨ ਪ੍ਰਦਾਨ ਕਰਨ ਦੀ ਲੋੜ ਹੈ ਜੋ ਆਫਸੈਟ ਵੇਕਟਰ ਤਿਆਰ ਕਰੇਗਾ:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

ਨੋਟ ਕਰੋ, ਕਿ ਪਿਛਲੇ ਸਾਰੇ ਉਦਾਹਰਣਾਂ ਤੋਂ ਵੱਖਰਾ, ਹੁਣ ਸਾਡਾ ਨੈਟਵਰਕ ਦੋ ਪੈਰਾਮੀਟਰਾਂ ਨੂੰ ਸਵੀਕਾਰ ਕਰਦਾ ਹੈ: ਡਾਟਾ ਵੈਕਟਰ ਅਤੇ ਆਫਸੈਟ ਵੈਕਟਰ, ਜੋ ਵੱਖ-ਵੱਖ ਆਕਾਰ ਦੇ ਹਨ। ਇਸੇ ਤਰ੍ਹਾਂ, ਸਾਡਾ ਡਾਟਾ ਲੋਡਰ ਵੀ ਸਾਨੂੰ 2 ਦੀ ਬਜਾਏ 3 ਮੁੱਲ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ: ਦੋਵੇਂ ਟੈਕਸਟ ਅਤੇ ਆਫਸੈਟ ਵੈਕਟਰ ਫੀਚਰਾਂ ਵਜੋਂ ਪ੍ਰਦਾਨ ਕੀਤੇ ਜਾਂਦੇ ਹਨ। ਇਸ ਲਈ, ਸਾਨੂੰ ਆਪਣੀ ਟ੍ਰੇਨਿੰਗ ਫੰਕਸ਼ਨ ਨੂੰ ਥੋੜ੍ਹਾ ਸਮਰਥਨ ਕਰਨ ਦੀ ਲੋੜ ਹੈ ਤਾਂ ਕਿ ਇਸਦਾ ਧਿਆਨ ਰੱਖਿਆ ਜਾ ਸਕੇ:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## ਸੈਮੈਂਟਿਕ ਐਮਬੈਡਿੰਗਸ: ਵਰਡ2ਵੈਕ

ਸਾਡੇ ਪਿਛਲੇ ਉਦਾਹਰਨ ਵਿੱਚ, ਮਾਡਲ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਨੇ ਸ਼ਬਦਾਂ ਨੂੰ ਵੈਕਟਰ ਪ੍ਰਤੀਨਿਧੀ ਵਿੱਚ ਮੈਪ ਕਰਨਾ ਸਿੱਖਿਆ, ਪਰ ਇਸ ਪ੍ਰਤੀਨਿਧੀ ਵਿੱਚ ਜ਼ਿਆਦਾ ਸੈਮੈਂਟਿਕ ਅਰਥ ਨਹੀਂ ਸੀ। ਇਹ ਵਧੀਆ ਹੋਵੇਗਾ ਕਿ ਅਜਿਹੇ ਵੈਕਟਰ ਪ੍ਰਤੀਨਿਧੀ ਸਿੱਖੇ ਜਾਣ, ਜਿੱਥੇ ਸਮਾਨ ਸ਼ਬਦ ਜਾਂ ਪਰਿਆਇ ਵਾਕ ਸ਼ਬਦ ਕੁਝ ਵੈਕਟਰ ਦੂਰੀ (ਜਿਵੇਂ ਕਿ ਯੂਕਲਿਡੀਅਨ ਦੂਰੀ) ਦੇ ਹਿਸਾਬ ਨਾਲ ਇੱਕ ਦੂਜੇ ਦੇ ਨੇੜੇ ਹੋਣ।

ਇਸ ਨੂੰ ਹਾਸਲ ਕਰਨ ਲਈ, ਸਾਨੂੰ ਆਪਣੇ ਐਮਬੈਡਿੰਗ ਮਾਡਲ ਨੂੰ ਇੱਕ ਵਿਸ਼ੇਸ਼ ਢੰਗ ਨਾਲ ਵੱਡੇ ਟੈਕਸਟ ਸੰਗ੍ਰਹਿ 'ਤੇ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕਰਨ ਦੀ ਲੋੜ ਹੈ। ਸੈਮੈਂਟਿਕ ਐਮਬੈਡਿੰਗਸ ਨੂੰ ਟ੍ਰੇਨ ਕਰਨ ਦੇ ਪਹਿਲੇ ਤਰੀਕਿਆਂ ਵਿੱਚੋਂ ਇੱਕ ਨੂੰ [ਵਰਡ2ਵੈਕ](https://en.wikipedia.org/wiki/Word2vec) ਕਿਹਾ ਜਾਂਦਾ ਹੈ। ਇਹ ਦੋ ਮੁੱਖ ਆਰਕੀਟੈਕਚਰਾਂ 'ਤੇ ਆਧਾਰਿਤ ਹੈ ਜੋ ਸ਼ਬਦਾਂ ਦੀ ਵੰਡਵਾਂ ਪ੍ਰਤੀਨਿਧੀ ਪੈਦਾ ਕਰਨ ਲਈ ਵਰਤੀ ਜਾਂਦੀ ਹੈ:

- **ਕੰਟਿਨਿਊਅਸ ਬੈਗ-ਆਫ-ਵਰਡਸ** (CBoW) — ਇਸ ਆਰਕੀਟੈਕਚਰ ਵਿੱਚ, ਅਸੀਂ ਮਾਡਲ ਨੂੰ ਆਸਪਾਸ ਦੇ ਸੰਦਰਭ ਤੋਂ ਇੱਕ ਸ਼ਬਦ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰਨ ਲਈ ਟ੍ਰੇਨ ਕਰਦੇ ਹਾਂ। ਦਿੱਤੇ ਗਏ ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, ਮਾਡਲ ਦਾ ਉਦੇਸ਼ $(W_{-2},W_{-1},W_1,W_2)$ ਤੋਂ $W_0$ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕਰਨਾ ਹੈ।
- **ਕੰਟਿਨਿਊਅਸ ਸਕਿਪ-ਗ੍ਰਾਮ** CBoW ਦੇ ਉਲਟ ਹੈ। ਮਾਡਲ ਆਸਪਾਸ ਦੇ ਸੰਦਰਭ ਸ਼ਬਦਾਂ ਦੀ ਵਿੰਡੋ ਨੂੰ ਵਰਤਦਾ ਹੈ ਤਾਕਿ ਮੌਜੂਦਾ ਸ਼ਬਦ ਦੀ ਭਵਿੱਖਵਾਣੀ ਕੀਤੀ ਜਾ ਸਕੇ।

CBoW ਤੇਜ਼ ਹੈ, ਜਦਕਿ ਸਕਿਪ-ਗ੍ਰਾਮ ਹੌਲੀ ਹੈ, ਪਰ ਇਹ ਅਲਭ ਸ਼ਬਦਾਂ ਦੀ ਵਧੀਆ ਪ੍ਰਤੀਨਿਧੀ ਕਰਦਾ ਹੈ।

![ਦੋਵੇਂ CBoW ਅਤੇ ਸਕਿਪ-ਗ੍ਰਾਮ ਐਲਗੋਰਿਥਮਾਂ ਨੂੰ ਸ਼ਬਦਾਂ ਨੂੰ ਵੈਕਟਰ ਵਿੱਚ ਬਦਲਣ ਲਈ ਦਿਖਾਉਣ ਵਾਲੀ ਚਿੱਤਰ।](../../../../../translated_images/pa/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

ਗੂਗਲ ਨਿਊਜ਼ ਡੇਟਾਸੈਟ 'ਤੇ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੇ ਵਰਡ2ਵੈਕ ਐਮਬੈਡਿੰਗ ਨਾਲ ਪ੍ਰਯੋਗ ਕਰਨ ਲਈ, ਅਸੀਂ **gensim** ਲਾਇਬ੍ਰੇਰੀ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹਾਂ। ਹੇਠਾਂ ਅਸੀਂ 'neural' ਦੇ ਸਭ ਤੋਂ ਸਮਾਨ ਸ਼ਬਦਾਂ ਨੂੰ ਲੱਭਦੇ ਹਾਂ।

> **Note:** ਜਦੋਂ ਤੁਸੀਂ ਪਹਿਲੀ ਵਾਰ ਸ਼ਬਦ ਵੈਕਟਰ ਬਣਾਉਂਦੇ ਹੋ, ਤਾਂ ਇਹਨਾਂ ਨੂੰ ਡਾਊਨਲੋਡ ਕਰਨ ਵਿੱਚ ਕੁਝ ਸਮਾਂ ਲੱਗ ਸਕਦਾ ਹੈ!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ਅਸੀਂ ਸ਼ਬਦ ਤੋਂ ਵੀਕਟਰ ਐਮਬੈਡਿੰਗਜ਼ ਦੀ ਗਣਨਾ ਕਰ ਸਕਦੇ ਹਾਂ, ਜੋ ਕਿ ਕਲਾਸੀਫਿਕੇਸ਼ਨ ਮਾਡਲ ਦੀ ਟ੍ਰੇਨਿੰਗ ਵਿੱਚ ਵਰਤੀ ਜਾਣੀ ਹੈ (ਸਪਸ਼ਟਤਾ ਲਈ ਅਸੀਂ ਸਿਰਫ਼ ਵੀਕਟਰ ਦੇ ਪਹਿਲੇ 20 ਘਟਕੇ ਦਿਖਾਉਂਦੇ ਹਾਂ):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

ਸਮਾਂਤਿਕਲ ਐਮਬੈਡਿੰਗਜ਼ ਬਾਰੇ ਵਧੀਆ ਗੱਲ ਇਹ ਹੈ ਕਿ ਤੁਸੀਂ ਸਿਮਾਂਤਿਕਸ ਨੂੰ ਬਦਲਣ ਲਈ ਵੈਕਟਰ ਕੋਡਿੰਗ ਨੂੰ ਮੋੜ ਸਕਦੇ ਹੋ। ਉਦਾਹਰਨ ਵਜੋਂ, ਅਸੀਂ ਇੱਕ ਸ਼ਬਦ ਲੱਭਣ ਲਈ ਪੁੱਛ ਸਕਦੇ ਹਾਂ, ਜਿਸਦਾ ਵੈਕਟਰ ਪ੍ਰਤੀਨਿਧਿਤਾ ਸ਼ਬਦ *king* ਅਤੇ *woman* ਦੇ ਜਿੰਨਾ ਨਜ਼ਦੀਕ ਹੋ ਸਕੇ ਹੋਵੇ, ਅਤੇ ਸ਼ਬਦ *man* ਤੋਂ ਜਿੰਨਾ ਦੂਰ ਹੋ ਸਕੇ:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

ਦੋਵੇਂ CBoW ਅਤੇ Skip-Grams "ਪ੍ਰਿਡਿਕਟਿਵ" ਐਮਬੈਡਿੰਗ ਹਨ, ਕਿਉਂਕਿ ਇਹ ਸਿਰਫ਼ ਸਥਾਨਕ ਸੰਦਰਭਾਂ ਨੂੰ ਹੀ ਧਿਆਨ ਵਿੱਚ ਰੱਖਦੇ ਹਨ। Word2Vec ਗਲੋਬਲ ਸੰਦਰਭ ਦਾ ਫਾਇਦਾ ਨਹੀਂ ਲੈਂਦਾ।

**FastText**, Word2Vec 'ਤੇ ਆਧਾਰਿਤ ਹੈ, ਜੋ ਹਰ ਸ਼ਬਦ ਅਤੇ ਉਸ ਵਿੱਚ ਪਾਏ ਜਾਣ ਵਾਲੇ ਅੱਖਰ n-grams ਲਈ ਵੈਕਟਰ ਪ੍ਰਸਤੁਤੀਆਂ ਸਿੱਖਦਾ ਹੈ। ਪ੍ਰਸਤੁਤੀਆਂ ਦੇ ਮੁੱਲ ਹਰ ਟ੍ਰੇਨਿੰਗ ਕਦਮ 'ਤੇ ਇੱਕ ਵੈਕਟਰ ਵਿੱਚ ਔਸਤ ਕੀਤੇ ਜਾਂਦੇ ਹਨ। ਹਾਲਾਂਕਿ ਇਹ ਪ੍ਰੀ-ਟ੍ਰੇਨਿੰਗ ਵਿੱਚ ਕਾਫ਼ੀ ਵਾਧੂ ਗਣਨਾ ਜੋੜਦਾ ਹੈ, ਪਰ ਇਹ ਸ਼ਬਦ ਐਮਬੈਡਿੰਗ ਨੂੰ ਉਪ-ਸ਼ਬਦ ਜਾਣਕਾਰੀ ਨੂੰ ਕੋਡ ਕਰਨ ਯੋਗ ਬਣਾਉਂਦਾ ਹੈ।

ਇਕ ਹੋਰ ਵਿਧੀ, **GloVe**, ਸਹ-ਘਟਨਾ ਮੈਟ੍ਰਿਕਸ ਦੇ ਵਿਚਾਰ ਨੂੰ ਲਾਭਦਾਇਕ ਬਣਾਉਂਦੀ ਹੈ ਅਤੇ ਸਹ-ਘਟਨਾ ਮੈਟ੍ਰਿਕਸ ਨੂੰ ਹੋਰ ਪ੍ਰਗਟਾਵਕ ਅਤੇ ਗੈਰ-ਰੇਖਿਕ ਸ਼ਬਦ ਵੈਕਟਰਾਂ ਵਿੱਚ ਵਿਘਟਿਤ ਕਰਨ ਲਈ ਨਿਊਰਲ ਤਰੀਕੇ ਵਰਤਦੀ ਹੈ।

ਤੁਹਾਨੂੰ FastText ਅਤੇ GloVe ਐਮਬੈਡਿੰਗ ਨੂੰ ਬਦਲ ਕੇ ਉਦਾਹਰਣ ਨਾਲ ਖੇਡਣ ਦਾ ਮੌਕਾ ਮਿਲਦਾ ਹੈ, ਕਿਉਂਕਿ gensim ਕਈ ਵੱਖ-ਵੱਖ ਸ਼ਬਦ ਐਮਬੈਡਿੰਗ ਮਾਡਲਾਂ ਦਾ ਸਮਰਥਨ ਕਰਦਾ ਹੈ।


## ਪਾਈਟਾਰਚ ਵਿੱਚ ਪਹਿਲਾਂ ਤੋਂ ਤਿਆਰ ਕੀਤੇ ਗਏ ਐਮਬੈਡਿੰਗਜ਼ ਦੀ ਵਰਤੋਂ

ਅਸੀਂ ਉਪਰੋਕਤ ਉਦਾਹਰਨ ਨੂੰ ਇਸ ਤਰ੍ਹਾਂ ਬਦਲ ਸਕਦੇ ਹਾਂ ਕਿ ਆਪਣੇ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਵਿੱਚ ਮੈਟ੍ਰਿਕਸ ਨੂੰ ਸੈਮਾਂਟਿਕ ਐਮਬੈਡਿੰਗਜ਼, ਜਿਵੇਂ ਕਿ Word2Vec ਨਾਲ ਪਹਿਲਾਂ ਤੋਂ ਭਰ ਸਕੀਏ। ਸਾਨੂੰ ਇਹ ਧਿਆਨ ਵਿੱਚ ਰੱਖਣਾ ਪਵੇਗਾ ਕਿ ਪਹਿਲਾਂ ਤੋਂ ਤਿਆਰ ਕੀਤੇ ਗਏ ਐਮਬੈਡਿੰਗ ਅਤੇ ਸਾਡੇ ਟੈਕਸਟ ਕੋਰਪਸ ਦੇ ਵੋਕੈਬੁਲਰੀਜ਼ ਅਕਸਰ ਮਿਲਦੇ ਨਹੀਂ ਹਨ, ਇਸ ਲਈ ਅਸੀਂ ਗੁੰਮ ਹੋਏ ਸ਼ਬਦਾਂ ਲਈ ਵਜ਼ਨ ਨੂੰ ਰੈਂਡਮ ਵੈਲਿਊਜ਼ ਨਾਲ ਸ਼ੁਰੂ ਕਰਾਂਗੇ:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


ਹੁਣ ਆਓ ਅਸੀਂ ਆਪਣੇ ਮਾਡਲ ਨੂੰ ਟ੍ਰੇਨ ਕਰੀਏ। ਧਿਆਨ ਦਿਓ ਕਿ ਮਾਡਲ ਨੂੰ ਟ੍ਰੇਨ ਕਰਨ ਵਿੱਚ ਲੱਗਣ ਵਾਲਾ ਸਮਾਂ ਪਿਛਲੇ ਉਦਾਹਰਨ ਦੇ ਮੁਕਾਬਲੇ ਕਾਫ਼ੀ ਵੱਧ ਹੈ, ਵੱਡੇ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਸਾਈਜ਼ ਦੇ ਕਾਰਨ, ਅਤੇ ਇਸ ਤਰ੍ਹਾਂ ਕਾਫ਼ੀ ਵੱਧ ਪੈਰਾਮੀਟਰਾਂ ਦੀ ਗਿਣਤੀ। ਇਸ ਦੇ ਨਾਲ ਹੀ, ਇਸ ਕਾਰਨ, ਜੇਕਰ ਅਸੀਂ ਓਵਰਫਿਟਿੰਗ ਤੋਂ ਬਚਣਾ ਚਾਹੁੰਦੇ ਹਾਂ ਤਾਂ ਸਾਨੂੰ ਆਪਣੇ ਮਾਡਲ ਨੂੰ ਹੋਰ ਉਦਾਹਰਨਾਂ 'ਤੇ ਟ੍ਰੇਨ ਕਰਨ ਦੀ ਲੋੜ ਹੋ ਸਕਦੀ ਹੈ।


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

ਸਾਡੇ ਮਾਮਲੇ ਵਿੱਚ, ਸਾਨੂੰ ਸ਼ੁੱਧਤਾ ਵਿੱਚ ਵੱਡਾ ਵਾਧਾ ਨਹੀਂ ਦਿਖਾਈ ਦਿੰਦਾ, ਜੋ ਸੰਭਵਤ: ਬਹੁਤ ਵੱਖਰੇ ਸ਼ਬਦਾਵਲੀ ਕਾਰਨ ਹੈ।  
ਵੱਖਰੀ ਸ਼ਬਦਾਵਲੀ ਦੀ ਸਮੱਸਿਆ ਨੂੰ ਹੱਲ ਕਰਨ ਲਈ, ਅਸੀਂ ਹੇਠਾਂ ਦਿੱਤੇ ਹੱਲਾਂ ਵਿੱਚੋਂ ਇੱਕ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹਾਂ:  
* ਆਪਣੇ ਸ਼ਬਦਾਵਲੀ 'ਤੇ word2vec ਮਾਡਲ ਨੂੰ ਮੁੜ-ਟ੍ਰੇਨ ਕਰੋ  
* ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੇ word2vec ਮਾਡਲ ਦੀ ਸ਼ਬਦਾਵਲੀ ਨਾਲ ਸਾਡਾ ਡਾਟਾਸੈਟ ਲੋਡ ਕਰੋ। ਡਾਟਾਸੈਟ ਨੂੰ ਲੋਡ ਕਰਨ ਲਈ ਵਰਤੀ ਗਈ ਸ਼ਬਦਾਵਲੀ ਨੂੰ ਲੋਡਿੰਗ ਦੌਰਾਨ ਨਿਰਧਾਰਤ ਕੀਤਾ ਜਾ ਸਕਦਾ ਹੈ।  

ਦੂਜਾ ਤਰੀਕਾ ਆਸਾਨ ਲੱਗਦਾ ਹੈ, ਖਾਸ ਕਰਕੇ ਕਿਉਂਕਿ PyTorch `torchtext` ਫਰੇਮਵਰਕ ਵਿੱਚ embeddings ਲਈ ਬਿਲਟ-ਇਨ ਸਹਾਇਤਾ ਹੈ।  
ਉਦਾਹਰਣ ਵਜੋਂ, ਅਸੀਂ ਹੇਠਾਂ ਦਿੱਤੇ ਤਰੀਕੇ ਨਾਲ GloVe-ਅਧਾਰਿਤ ਸ਼ਬਦਾਵਲੀ ਨੂੰ ਇੰਸਟੈਂਸ਼ੀਏਟ ਕਰ ਸਕਦੇ ਹਾਂ:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


ਲੋਡ ਕੀਤੇ ਸ਼ਬਦ-ਭੰਡਾਰ ਵਿੱਚ ਹੇਠ ਲਿਖੀਆਂ ਮੁੱਢਲੀਆਂ ਕਾਰਵਾਈਆਂ ਸ਼ਾਮਲ ਹਨ:
* `vocab.stoi` ਡਿਕਸ਼ਨਰੀ ਸਾਨੂੰ ਸ਼ਬਦ ਨੂੰ ਇਸਦੇ ਡਿਕਸ਼ਨਰੀ ਇੰਡੈਕਸ ਵਿੱਚ ਬਦਲਣ ਦੀ ਆਗਿਆ ਦਿੰਦੀ ਹੈ
* `vocab.itos` ਇਸਦਾ ਉਲਟ ਕਰਦਾ ਹੈ - ਨੰਬਰ ਨੂੰ ਸ਼ਬਦ ਵਿੱਚ ਬਦਲਦਾ ਹੈ
* `vocab.vectors` ਐਮਬੈਡਿੰਗ ਵੇਕਟਰਾਂ ਦੀ ਐਰੇ ਹੈ, ਇਸ ਲਈ ਕਿਸੇ ਸ਼ਬਦ `s` ਦੀ ਐਮਬੈਡਿੰਗ ਪ੍ਰਾਪਤ ਕਰਨ ਲਈ ਸਾਨੂੰ `vocab.vectors[vocab.stoi[s]]` ਵਰਤਣਾ ਪਵੇਗਾ

ਇੱਥੇ ਐਮਬੈਡਿੰਗਜ਼ ਨਾਲ ਹੇਰਫੇਰ ਕਰਨ ਦਾ ਉਦਾਹਰਨ ਦਿੱਤਾ ਗਿਆ ਹੈ ਤਾਂ ਜੋ ਸਮੀਕਰਨ **kind-man+woman = queen** ਨੂੰ ਦਰਸਾਇਆ ਜਾ ਸਕੇ (ਮੈਨੂੰ ਇਸਨੂੰ ਕੰਮ ਕਰਨ ਲਈ ਗੁਣਾਂਕ ਵਿੱਚ ਥੋੜ੍ਹਾ ਬਦਲਾਅ ਕਰਨਾ ਪਿਆ):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

ਕਲਾਸੀਫਾਇਰ ਨੂੰ ਉਹਨਾਂ ਐਮਬੈਡਿੰਗਜ਼ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਟ੍ਰੇਨ ਕਰਨ ਲਈ, ਸਾਨੂੰ ਪਹਿਲਾਂ ਆਪਣਾ ਡੇਟਾਸੈੱਟ GloVe ਸ਼ਬਦਾਵਲੀ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਐਨਕੋਡ ਕਰਨ ਦੀ ਲੋੜ ਹੈ:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

ਜਿਵੇਂ ਕਿ ਅਸੀਂ ਉੱਪਰ ਦੇਖਿਆ ਹੈ, ਸਾਰੇ ਵੇਕਟਰ ਐਮਬੈਡਿੰਗਜ਼ `vocab.vectors` ਮੈਟ੍ਰਿਕਸ ਵਿੱਚ ਸਟੋਰ ਕੀਤੇ ਜਾਂਦੇ ਹਨ। ਇਹਨਾਂ ਵਜ਼ਨਾਂ ਨੂੰ ਸਧਾਰਨ ਕਾਪੀ ਕਰਨ ਦੁਆਰਾ ਐਮਬੈਡਿੰਗ ਲੇਅਰ ਦੇ ਵਜ਼ਨਾਂ ਵਿੱਚ ਲੋਡ ਕਰਨਾ ਬਹੁਤ ਆਸਾਨ ਬਣਾ ਦਿੰਦਾ ਹੈ:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

ਹੇਠਾਂ ਦਿੱਤੇ ਕਾਰਨਾਂ ਵਿੱਚੋਂ ਇੱਕ ਇਹ ਹੈ ਕਿ ਸਾਨੂੰ ਸ਼ੁੱਧਤਾ ਵਿੱਚ ਮਹੱਤਵਪੂਰਨ ਵਾਧਾ ਨਹੀਂ ਦਿਖਾਈ ਦੇ ਰਿਹਾ ਹੈ ਕਿਉਂਕਿ ਸਾਡੇ ਡੇਟਾਸੈਟ ਦੇ ਕੁਝ ਸ਼ਬਦ ਪਹਿਲਾਂ ਤੋਂ ਪ੍ਰਸ਼ਿਕਸ਼ਿਤ GloVe ਸ਼ਬਦਾਵਲੀ ਵਿੱਚ ਮੌਜੂਦ ਨਹੀਂ ਹਨ, ਅਤੇ ਇਸ ਲਈ ਉਹ ਅਸਲ ਵਿੱਚ ਅਣਗੌਰ ਕੀਤੇ ਜਾਂਦੇ ਹਨ। ਇਸ ਤੱਥ ਨੂੰ ਦੂਰ ਕਰਨ ਲਈ, ਅਸੀਂ ਆਪਣੇ ਡੇਟਾਸੈਟ 'ਤੇ ਆਪਣੇ ਐਮਬੈਡਿੰਗਜ਼ ਨੂੰ ਪ੍ਰਸ਼ਿਕਸ਼ਿਤ ਕਰ ਸਕਦੇ ਹਾਂ।


## ਸੰਦਰਭਿਕ ਐਮਬੈਡਿੰਗਸ

ਪ੍ਰੰਪਰਾਗਤ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੀਆਂ ਐਮਬੈਡਿੰਗ ਪ੍ਰਸਤੁਤੀਆਂ ਜਿਵੇਂ ਕਿ Word2Vec ਦੀ ਇੱਕ ਮੁੱਖ ਸੀਮਿਤਤਾ ਸ਼ਬਦ ਅਰਥ ਸਪਸ਼ਟੀਕਰਨ ਦੀ ਸਮੱਸਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੀਆਂ ਐਮਬੈਡਿੰਗਸ ਸ਼ਬਦਾਂ ਦੇ ਸੰਦਰਭ ਵਿੱਚ ਕੁਝ ਅਰਥ ਨੂੰ ਕੈਪਚਰ ਕਰ ਸਕਦੀਆਂ ਹਨ, ਇੱਕ ਸ਼ਬਦ ਦੇ ਹਰ ਸੰਭਾਵਿਤ ਅਰਥ ਨੂੰ ਇੱਕੋ ਐਮਬੈਡਿੰਗ ਵਿੱਚ ਕੋਡ ਕੀਤਾ ਜਾਂਦਾ ਹੈ। ਇਹ ਹੇਠਾਂ ਦਿੱਤੇ ਮਾਡਲਾਂ ਵਿੱਚ ਸਮੱਸਿਆ ਪੈਦਾ ਕਰ ਸਕਦਾ ਹੈ, ਕਿਉਂਕਿ ਬਹੁਤ ਸਾਰੇ ਸ਼ਬਦ, ਜਿਵੇਂ 'play', ਦਾ ਅਰਥ ਸੰਦਰਭ ਦੇ ਅਨੁਸਾਰ ਵੱਖ-ਵੱਖ ਹੋ ਸਕਦਾ ਹੈ।

ਉਦਾਹਰਣ ਲਈ, 'play' ਸ਼ਬਦ ਦਾ ਅਰਥ ਹੇਠਾਂ ਦਿੱਤੇ ਦੋ ਵਾਕਾਂ ਵਿੱਚ ਬਹੁਤ ਵੱਖਰਾ ਹੈ:
- ਮੈਂ ਥੀਏਟਰ ਵਿੱਚ ਇੱਕ **play** ਦੇਖਣ ਗਿਆ।
- ਜੌਨ ਆਪਣੇ ਦੋਸਤਾਂ ਨਾਲ **play** ਕਰਨਾ ਚਾਹੁੰਦਾ ਹੈ।

ਉਪਰੋਕਤ ਪ੍ਰੀ-ਟ੍ਰੇਨ ਕੀਤੀਆਂ ਐਮਬੈਡਿੰਗਸ 'play' ਸ਼ਬਦ ਦੇ ਦੋਨੋ ਅਰਥਾਂ ਨੂੰ ਇੱਕੋ ਐਮਬੈਡਿੰਗ ਵਿੱਚ ਦਰਸਾਉਂਦੀਆਂ ਹਨ। ਇਸ ਸੀਮਿਤਤਾ ਨੂੰ ਦੂਰ ਕਰਨ ਲਈ, ਸਾਨੂੰ **ਭਾਸ਼ਾ ਮਾਡਲ** ਦੇ ਆਧਾਰ 'ਤੇ ਐਮਬੈਡਿੰਗ ਬਣਾਉਣ ਦੀ ਲੋੜ ਹੈ, ਜੋ ਕਿ ਵੱਡੇ ਟੈਕਸਟ ਕੋਰਪਸ 'ਤੇ ਟ੍ਰੇਨ ਕੀਤਾ ਜਾਂਦਾ ਹੈ ਅਤੇ *ਜਾਣਦਾ ਹੈ* ਕਿ ਸ਼ਬਦਾਂ ਨੂੰ ਵੱਖ-ਵੱਖ ਸੰਦਰਭਾਂ ਵਿੱਚ ਕਿਵੇਂ ਜੋੜਿਆ ਜਾ ਸਕਦਾ ਹੈ। ਸੰਦਰਭਿਕ ਐਮਬੈਡਿੰਗਸ ਬਾਰੇ ਚਰਚਾ ਇਸ ਟਿਊਟੋਰਿਅਲ ਦੇ ਦਾਇਰੇ ਤੋਂ ਬਾਹਰ ਹੈ, ਪਰ ਅਸੀਂ ਇਸ ਬਾਰੇ ਅਗਲੇ ਯੂਨਿਟ ਵਿੱਚ ਭਾਸ਼ਾ ਮਾਡਲਾਂ ਦੀ ਗੱਲ ਕਰਦੇ ਸਮੇਂ ਵਾਪਸ ਆਵਾਂਗੇ।



---

**ਅਸਵੀਕਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ ਨੂੰ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀਤਾ ਲਈ ਯਤਨਸ਼ੀਲ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚਤਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
